In [7]:
%matplotlib qt
%load_ext autoreload
%autoreload 2
import napari
import matplotlib.pyplot as plt 
import numpy as np
from pathlib import Path

from utils import  load_image, select_elipse_from_stack
from consts import UMBRELLA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
coord_path = Path("/Users/jamesrowland/Code/dlx/coords")
rna = "90"
assert rna in {"90", "91", "scrambled"}
well = 1

experiments = list((UMBRELLA / rna / str(well)).glob("*.czi"))

In [9]:
viewer = napari.Viewer()

experiment = experiments[0]
file_name = f"{rna}_{well}_{experiment.name.strip(".czi")}.npy"
image = load_image(str(experiment))
stack = image.get_image_data().squeeze()

average = stack.mean(axis=0)

viewer.add_image(stack, name="full stack", scale=(1, 1, 1))
viewer.add_image(average, name="average")


<Image layer 'average' at 0x153d34260>

In [10]:
shapes_layers = [layer for layer in viewer.layers if "Shapes" in layer.name]
coords = [d for layer in shapes_layers for d in layer.data]
assert len(coords) > 0
np.save(coord_path / file_name, coords)

AssertionError: 

In [11]:
coordys = np.load(coord_path / file_name)
for idx, coord in enumerate(coordys):
    x = select_elipse_from_stack(coord, stack)
    viewer.add_image(x, name=f"elipse {idx}")

In [18]:
plt.plot(np.nanmean(x, axis=(1, 2)))

In [5]:
if experiment.name == "Experiment-1363.czi" and rna == "900000":
    corner_pixels = np.array([[  0, 675, 383],[  0, 766, 484]])
elif experiment.name == 'Experiment-1354_10x.czi' and rna == "scrambled":
    corner_pixels = np.array([[  0, 528, 477], [ 0, 607, 567]])
else:
    shape = [layer for layer in viewer.layers if layer.name == "Shapes"][0]
    assert len(shape.data) == 1
    corner_pixels = shape.corner_pixels


top_left =  [corner_pixels[0, 1], corner_pixels[0, 2]]
bottom_right = [corner_pixels[1,1], corner_pixels[1,2]]
cell = stack[:, top_left[0]: bottom_right[0], top_left[1]: bottom_right[1]]

viewer.add_image(cell, name="cell")

<Image layer 'cell' at 0x15550d130>

In [6]:
plt.clf()
x = np.linspace(0, 3 * 60, cell.shape[0])
plt.plot(x, np.mean(cell, axis=(1,2)))
plt.plot(x, np.mean(cell, axis=(1,2)), '.') 

plt.xlabel("Time (s)")

plt.savefig(Path("/Users/jamesrowland/Code/dlx/figures") / f"{rna}_{well}.png")